In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms,models
from matplotlib import pyplot as plt
from matplotlib import ticker

In [2]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Forward pass
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculate metrics
        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        
        if batch % 10 == 0:
            print(f"loss: {loss.item():>7f}  [{batch * len(X):>5d}/{size:>5d}]")
    
    train_loss /= num_batches
    correct /= size
    
    train_Avg_loss.append(train_loss)
    train_Accuracy.append(100 * correct)
    print(f"Training Accuracy: {100 * correct:>0.1f}%")

def test(dataloader,model,loss_fn):
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    model.eval()
    test_loss,correct=0,0
    with torch.no_grad():
        for X,y in dataloader:
            X, y = X.to(device), y.to(device)
            pred=model(X)
            test_loss+=loss_fn(pred,y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss/=num_batches
    correct/=size
    
    test_Avg_loss.append(test_loss)
    test_Accuracy.append(100*correct)
    
    print(f"Test Accuracy: {(100*correct):>0.2f}%, Test Avg loss: {test_loss:>8f}\n")

In [3]:
transform1=transforms.Compose([transforms.Resize((128,128)),
                               transforms.RandomHorizontalFlip(p=0.5),
                               #transforms.Lambda(lambda x: x.convert('RGBA') if x.mode == 'P' else x),
                               transforms.ToTensor(),
                               transforms.Normalize(0,1)])
transform2=transforms.Compose([transforms.Resize((128,128)),
                               #transforms.Lambda(lambda x: x.convert('RGBA') if x.mode == 'P' else x),
                               transforms.ToTensor(),
                               transforms.Normalize(0,1)])

In [ ]:
training_data=datasets.ImageFolder(r'datasets/fruit_and_vegetable/train',transform=transform1)
test_data=datasets.ImageFolder(r'datasets/fruit_and_vegetable/test',transform=transform2)
train_dataloader=DataLoader(training_data,batch_size=64,shuffle=True)
test_dataloader=DataLoader(test_data,batch_size=64,shuffle=True)

for X,y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
loss_fn=nn.CrossEntropyLoss()
learning_rate=1e-2

model=models.resnet18()
model.to(device)
#model.classifier[-1].out_features = 36

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.8)

train_Avg_loss = []
train_Accuracy = []
test_Accuracy = []
test_Avg_loss = []

epochs=10
for epoch in range(epochs):
    print(f"epoch{epoch+1}")
    train(train_dataloader,model,loss_fn,optimizer)
    scheduler.step()
    test(test_dataloader,model,loss_fn)
print('Done')

Shape of X [N, C, H, W]: torch.Size([64, 3, 128, 128])
Shape of y: torch.Size([64]) torch.int64
Using device: cuda
epoch1
loss: 6.903142  [    0/ 3115]
loss: 4.363432  [  640/ 3115]
loss: 3.915761  [ 1280/ 3115]
loss: 3.744495  [ 1920/ 3115]
loss: 3.554277  [ 2560/ 3115]
Training Accuracy: 7.6%
Test Accuracy: 14.76%, Test Avg loss: 3.215877

epoch2
loss: 3.204198  [    0/ 3115]
loss: 2.709663  [  640/ 3115]
loss: 2.891295  [ 1280/ 3115]
loss: 2.963515  [ 1920/ 3115]
loss: 2.715539  [ 2560/ 3115]
Training Accuracy: 16.5%
Test Accuracy: 15.32%, Test Avg loss: 2.907804

epoch3
loss: 2.571241  [    0/ 3115]
loss: 2.751004  [  640/ 3115]
loss: 2.435958  [ 1280/ 3115]
loss: 2.696160  [ 1920/ 3115]
loss: 2.779417  [ 2560/ 3115]
Training Accuracy: 20.6%
Test Accuracy: 28.13%, Test Avg loss: 2.316028

epoch4
loss: 2.459750  [    0/ 3115]
loss: 2.285815  [  640/ 3115]
loss: 2.456947  [ 1280/ 3115]
loss: 2.473643  [ 1920/ 3115]
loss: 2.591402  [ 2560/ 3115]
Training Accuracy: 24.7%
Test Accuracy: